In [18]:
from music21 import converter, instrument, note, chord
import glob
import numpy


In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

# Training

In [21]:
notes = []

n_vocab = len(set(notes))

for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    
    parts = instrument.partitionByInstrument(midi)
    
    if parts: #the notes are instrument whole notes to parse
        notes_to_parse = parts.parts[0].recurse()
    else: # if the file has notes in a flat structure
        notes_to_parse.flat.notes
    
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [22]:
#mapping the string based categorical data into an integer-based numerical data

# Create input sequences for hte network and their respective outputs

sequence_length = 100

# get all pitch names 
pitchnames = sorted(set(item for item in notes))

# create a dictionary to map pitches to integers

note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

network_input = []
network_output = []
                   

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
    
n_patterns = len(network_output)

#reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))

#normalize iput

network_input = network_input/float(n_vocab)

network_output = np_utils.to_categorical(network_output)

        

/Users/revanth/tf/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in true_divide
/Users/revanth/tf/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


# Model

In [23]:
# LSTM layers is a RNN that takes a sequence as input and can return
# either seqeunces (return_sequences = True) or a matrix

In [24]:
# Dropout layers are a regularization otechnique that consists
# of setting a fraction of input units to 0 at each update
# during the training to prevent overfitting. The fraction is 
# determined by the parameter used with the layer.



In [25]:
# Dense layers or fully connected layers is a fully connected 
# neural network layer where each input node is connected to
# each output node.


In [26]:
# The Activation layer determines what activation function our
# neural network will use to calculate the output of a node.

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]),
              return_sequences = True
              ))

model.add(Droput(0.3))